In [1]:
import numpy as np
import mpi4py.MPI
import dolfinx

In [2]:
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.mesh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl

In [3]:
import viskex

We consider the model boundary value problem:
$$
\left\{
\begin{array}{l}
- u'' = 2, & x \in I= (0, 1),\\
u(0) = 0,\\
u(1) = 1.
\end{array}
\right.
$$

**Task 1: create a mesh.**

`dolfinx.mesh` provide some built-in functions to generate simple meshes, and in particular `create_unit_interval` for an equispaced mesh on the unit interval $I$.

 Create the uniform mesh with 10 cells:

In [4]:
# define uniform grid 
mesh = dolfinx.mesh.create_unit_interval(mpi4py.MPI.COMM_WORLD, 12)

Note that `dolfinx.mesh` requires that we supply the MPI-communicator. This is to specify how we would like the program to behave in parallel. With:
* MPI.COMM_WORLD we create a single mesh, whose data is distributed over the number of processors we would like to use.
* MPI.COMM_SELF we create a separate mesh on each processor

We can obtain an interactive plot of the domain using viskex

In [5]:
viskex.dolfinx.plot_mesh(mesh)

Widget(value='<iframe src="http://localhost:45561/index.html?ui=P_0x76a2758e1850_0&reconnect=auto" class="pyvi…

A **mesh**  is made by
*  a set of points: these are part of the mesh.geometry
*  a set of subintervals that connect them: these are part of the mesh.topology



(Note that `dolfinx` developers decided to store points as vectors in $\mathbb{R}^3$, regardless of the actual ambient space dimension!)

In [6]:
points = mesh.geometry.x
points

array([[0.        , 0.        , 0.        ],
       [0.08333333, 0.        , 0.        ],
       [0.16666667, 0.        , 0.        ],
       [0.25      , 0.        , 0.        ],
       [0.33333333, 0.        , 0.        ],
       [0.41666667, 0.        , 0.        ],
       [0.5       , 0.        , 0.        ],
       [0.58333333, 0.        , 0.        ],
       [0.66666667, 0.        , 0.        ],
       [0.75      , 0.        , 0.        ],
       [0.83333333, 0.        , 0.        ],
       [0.91666667, 0.        , 0.        ],
       [1.        , 0.        , 0.        ]])

In [7]:
connectivity_cells_to_vertices = mesh.topology.connectivity(mesh.topology.dim, 0)
connectivity_cells_to_vertices

<AdjacencyList> with 12 nodes
  0: [0 1 ]
  1: [1 2 ]
  2: [2 3 ]
  3: [3 4 ]
  4: [4 5 ]
  5: [5 6 ]
  6: [6 7 ]
  7: [7 8 ]
  8: [8 9 ]
  9: [9 10 ]
  10: [10 11 ]
  11: [11 12 ]

In [8]:
num_cells = len(connectivity_cells_to_vertices)
num_cells

12

We can have a look at each cell  by using a `for` loop. Each cell is assigned an unique ID and (in 1D) it is uniquely defined by two vertices, which correspond to the endpoints of the subinterval.

In [9]:
for c in range(num_cells):
    # Print the ID of the current cell
    print("Cell ID", c, "is defined by the following vertices:")
    # Print the vertices of the current cell
    for v in connectivity_cells_to_vertices.links(c):
        print("\t" + "Vertex ID", v, "is located at x =", points[v][0])

Cell ID 0 is defined by the following vertices:
	Vertex ID 0 is located at x = 0.0
	Vertex ID 1 is located at x = 0.08333333333333333
Cell ID 1 is defined by the following vertices:
	Vertex ID 1 is located at x = 0.08333333333333333
	Vertex ID 2 is located at x = 0.16666666666666666
Cell ID 2 is defined by the following vertices:
	Vertex ID 2 is located at x = 0.16666666666666666
	Vertex ID 3 is located at x = 0.25
Cell ID 3 is defined by the following vertices:
	Vertex ID 3 is located at x = 0.25
	Vertex ID 4 is located at x = 0.3333333333333333
Cell ID 4 is defined by the following vertices:
	Vertex ID 4 is located at x = 0.3333333333333333
	Vertex ID 5 is located at x = 0.41666666666666663
Cell ID 5 is defined by the following vertices:
	Vertex ID 5 is located at x = 0.41666666666666663
	Vertex ID 6 is located at x = 0.5
Cell ID 6 is defined by the following vertices:
	Vertex ID 6 is located at x = 0.5
	Vertex ID 7 is located at x = 0.5833333333333333
Cell ID 7 is defined by the fol

Next, we identify the IDs corresponding to boundary nodes. We use the

`dolfinx.mesh` function `locate_entities_boundary`. It requires the following inputs:
 * the first argument is the mesh,
 * the second argument represent the topological dimension of the mesh entities which we are interested in. In 1D, `mesh.topology.dim` is equal to 1, and entities of topological dimension 1 are the cells (subintervals), while `mesh.topology.dim - 1` is equal to 0, and entities of topological dimension 0 are the vertices of mesh.
 * the third argument is a condition (i.e., a function that returns either `True` or `False`) on the coordinates `x`, which are stored as a vector. Since we are interested in finding the vertex located at $x = 0$, we may think of using `x[0] == 0` as a condition: however, due to floating point arithmetic, it is safer to use $\left|x - 0\right| < \varepsilon$, where $\varepsilon$ is a small number, which may be written as `np.isclose(x[0], 0.0)`.

In [10]:
# Also the dimension is a topological info:
tdim = mesh.topology.dim
fdim = tdim - 1

left_boundary_entities = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: np.isclose(x[0], 0.0))


right_boundary_entities = dolfinx.mesh.locate_entities(
    mesh, fdim, lambda x: np.isclose(x[0], 1.0))

print(left_boundary_entities)
print(right_boundary_entities)

[0]
[12]


**Task 2: create FEM space.**

We define the finite element function space $V_h$ using $\mathbb{P}_2$ Lagrange elements.

This is obtained using the `FunctionSpace` class of `dolfinx.fem`.

The first argument specifies the mesh. The second the type of FE space. To define the standard (conforming) Lagrange elements we input `"CG"`. Using instead `"Lagrange"` or `"P"` yields the same space.

In [11]:
Vh = dolfinx.fem.functionspace(mesh, ("P", 2))

In [12]:
Vh

FunctionSpace(Mesh(blocked element (Basix element (P, interval, 1, gll_warped, unset, False, float64, []), (1,)), 0), Basix element (P, interval, 2, gll_warped, unset, False, float64, []))

In [13]:
Vh_dim = Vh.dofmap.index_map.size_local
Vh_dim

25

Once the FE space is at hand, we introduce *ufl*  (unified form language) symbols to define the trial and test functions for our weak formulation:

In [14]:
uh = ufl.TrialFunction(Vh)
vh = ufl.TestFunction(Vh)

In [15]:
uh

Argument(FunctionSpace(Mesh(blocked element (Basix element (P, interval, 1, gll_warped, unset, False, float64, []), (1,)), 0), Basix element (P, interval, 2, gll_warped, unset, False, float64, [])), 1, None)

**Task 3:** Set up FEM system

Now we are ready to define the FEM using the ufl capability.
* `uh.dx(0)` corresponds to $\frac{\partial u}{\partial x}$, where the argument `0` to `dx` means to take the derivative with respect to the first space coordinate (the only one of interest in this case).
* `ufl.dx` provides a measure for integration over the domain. Integration will automatically occur over the entire domain.

In [16]:
dx = ufl.dx

A = uh.dx(0) * vh.dx(0) * dx #bi-linear operator

F = 2 * vh * dx

**Task 4:** Apply boundary conditions

It remains to implement the boundary conditions. To do so we:
* determine the degree of freedom that corresponds to the boundary vertices.
* define a `Constant` equal to `0` and a `Constant` equal to `1` corresponding to the values on the boundary.
* create a list containing the Dirichlet boundary conditions (two in this case), that is the constraints on the FE function DoF:

We can help ourselves looking at the following table, which has in the first colum the ID of the degree of freedom, and in the second column the corresponding 𝑥 coordinate.

In [20]:
left_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim-1, left_boundary_entities)
right_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim-1, right_boundary_entities)

In [22]:
zero = dolfinx.fem.Constant(mesh, 0.)
one = dolfinx.fem.Constant(mesh, 1.)

In [24]:
bcs = [dolfinx.fem.dirichletbc(zero, left_boundary_dofs, Vh), dolfinx.fem.dirichletbc(one, right_boundary_dofs, Vh)]

In [25]:
bcs

**Task 5:** Solve the FEM system

In order to solve the FEM system, we go through the following steps:

* `dolfinx.fem` provides a `Function` class to store the solution of a finite element problem:
* solve the discrete problem allocating a new `LinearProblem` (which uses `PETSc`), providing as input the bilinear form `a`, the linear functional `F`, the boundary conditions `bcs`, and where to store the solution. Further solver options can also be passed to `PETSc`.

In [ ]:
solution = dolfinx.fem.Function(Vh)

In [37]:
problem = dolfinx.fem.petsc.LinearProblem(
    A, F, bcs=bcs, u=solution,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()

**Task 6:** compute the $L^2$ and $H^1$ errors.

The exact solution is:
$$ u(x) = - x^2 + 2 x.$$

The $L^2(I)$ norm of the error $u - u_h$ is defined as:
$$ e_h^2 = \int_I \left(u(x) - u_h(x)\right)^2 \ \mathrm{d}x.$$

In [41]:
xyz = ufl.SpatialCoordinate(mesh)
x = xyz[0]

In [43]:
exact_solution = - x**2 + 2 * x

In [44]:
error_L2squared_ufl = (exact_solution - solution)**2 * dx

Note that, given that we are using quadratic elements, we expect the error to be zero!

In [45]:
error_L2squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(error_L2squared_ufl))
print(error_L2squared)

4.748298230819104e-29


In [46]:
error_H1squared_ufl = (exact_solution.dx(0) - solution.dx(0) ) **2 * dx
error_H1squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(error_H1squared_ufl))
error_H1squared

5.3487495367326724e-28